In [1]:
import os
import numpy as np
import copy
import random
import sys
import tensorflow as tf

if '../../' not in sys.path:
    sys.path.append('../../')
    
from cnnbench.lib import graph_util, module_spec, base_ops, model_builder
from cnnbench.lib import config as _config

config = _config.build_config()

is_training = True

if config['data_format'] == 'channels_last':
  channel_axis = 3
elif config['data_format'] == 'channels_first':
  # Currently this is not well supported
  channel_axis = 1
else:
  raise ValueError('invalid data_format')

In [2]:
config['available_ops']

['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3']

In [3]:
def get_model(graph):
    matrices_list = []
    labels_list = []
    spec_list = []

    for module in graph:
      matrix = np.array(module[0])

      # Re-label to config['available_ops']
      labels = (['input'] +
              [config['available_ops'][lab] for lab in module[1][1:-1]] +
              ['output'])

      spec = module_spec.ModuleSpec(matrix, labels, config['hash_algo'])

      assert spec.valid_spec
      assert np.sum(spec.matrix) <= config['max_edges'] 

      matrices_list.append(matrix)
      labels_list.append(labels)
      spec_list.append(spec)

    input = tf.keras.layers.Input(shape=(224, 224, 3))

    if config['run_nasbench']:
      # Initial stem convolution
      net = base_ops.conv_bn_relu(
          input, 3, config['stem_filter_size'],
          is_training, config['data_format'])

      channels = net.get_shape()[channel_axis]

      for module_num in range(len(graph)):
        spec = spec_list[module_num]
        net = model_builder.build_module(
          spec,
          inputs=net,
          channels=channels,
          is_training=is_training)

      net = tf.keras.layers.GlobalAvgPool2D()(net)

      output = tf.keras.layers.Dense(1000, activation='softmax')(net)


    model = tf.keras.Model(input, output)
    return model

In [9]:
mat1 = np.zeros((3, 3))
mat1[0, 1] = mat1[1, 2] = 1
lab1 = [-1, 0, -2]
mat2 = copy.deepcopy(mat1)
lab2 = [-1, 0, -2]

graph1 = [(mat1.astype(int).tolist(), lab1), (mat2.astype(int).tolist(), lab2)]

# mat1 = np.zeros((2, 2))
# mat1[0, 1] = 1
# lab1 = [-1, -2]
# graph1 = [(mat1, lab1)]

print('Graph:')
for i in range(len(graph1)):
    print(f'Matrix:\n {graph1[i][0]}')
    print(f'Labels: {graph1[i][1]}')

print(f'Graph hash: {graph_util.hash_graph(graph1)}')

model = get_model(graph1)
model.summary()

Graph:
Matrix:
 [[0, 1, 0], [0, 0, 1], [0, 0, 0]]
Labels: [-1, 0, -2]
Matrix:
 [[0, 1, 0], [0, 0, 1], [0, 0, 0]]
Labels: [-1, 0, -2]
Graph hash: f3c93a709b571b3c7a1b5482e5940b31
INFO:tensorflow:vertex_channels: [128, 128, 128]
INFO:tensorflow:vertex_channels: [128, 128, 128]
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 224, 224, 128)     3456      
_________________________________________________________________
batch_normalization_6 (Batch (None, 224, 224, 128)     512       
_________________________________________________________________
re_lu_6 (ReLU)               (None, 224, 224, 128)     0         
_________________________________________________________________
tf_op_layer_input_4 (TensorF [(None, 224, 224, 

In [10]:
mat3 = np.zeros((4, 4))
mat3[0, 1] = mat3[1, 2] = mat3[2, 3] = 1
lab3 = [-1, 0, 0, -2]

graph2 = [(mat3.astype(int).tolist(), lab3)]

print('Graph:')
for i in range(len(graph2)):
    print(f'Matrix:\n {graph2[i][0]}')
    print(f'Labels:\n {graph2[i][1]}')

print(f'Graph hash: {graph_util.hash_graph(graph2)}')

model = get_model(graph2)
model.summary()

Graph:
Matrix:
 [[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]]
Labels:
 [-1, 0, 0, -2]
Graph hash: f3c93a709b571b3c7a1b5482e5940b31
INFO:tensorflow:vertex_channels: [128, 128, 128, 128]
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 224, 224, 128)     3456      
_________________________________________________________________
batch_normalization_9 (Batch (None, 224, 224, 128)     512       
_________________________________________________________________
re_lu_9 (ReLU)               (None, 224, 224, 128)     0         
_________________________________________________________________
tf_op_layer_input_6 (TensorF [(None, 224, 224, 128)]   0         
_____________________________________________________________

In [11]:
model.count_params()

428904

In [6]:
graph_util.hash_graph_simple(graph1) == graph_util.hash_graph_simple(graph2) # Should be false

False

In [7]:
graph_util.hash_graph(graph1) == graph_util.hash_graph(graph2) # Should be true

True

In [8]:
graph_util.compare_graphs(graph1, graph2) # Should be true

True